In [3]:
!pip install -q torch torchvision numpy

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from collections import OrderedDict
import itertools

from prunenn import pruner

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pruning_layers = nn.Sequential(OrderedDict([
            ("Layer1",
                nn.Sequential(
                  nn.Conv2d(3, 64, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(64, track_running_stats=True)
                )),
            ("Layer2",
                nn.Sequential(
                  nn.Conv2d(64, 128, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(128, track_running_stats=True)
                )),
            ("Layer4",
                nn.Sequential(
                  nn.Conv2d(128, 256, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(256, track_running_stats=True)
                )),
            ("Layer5",
                nn.Sequential(
                  nn.Conv2d(256, 256, 5, padding=2),
                  nn.MaxPool2d(2),
                  nn.ReLU(), 
                  nn.BatchNorm2d(256, track_running_stats=True),
                  nn.Dropout(0.5)
                ))
            ])) 
        self.fc1 = nn.Linear(1024, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.pruning_layers(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return 1. * correct / len(test_loader.dataset)

Files already downloaded and verified


In [ ]:
model = Net()

device = torch.device("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)#, momentum=0.5)

In [0]:
import time

epoch_range = 5

for epoch in range(epoch_range):
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

pruning_model = Adder(model, thres = 0.5, function = "var")

pruning_model.to_train = False
t0 = time.time()
acc = test(pruning_model, device, val_loader)
t1 = time.time()
print("testing time", (t1-t0))


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.341164
Train Epoch: 0 [6400/50000 (13%)]	Loss: 1.489525
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.231428
Train Epoch: 0 [19200/50000 (38%)]	Loss: 1.090029
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.114900
Train Epoch: 0 [32000/50000 (64%)]	Loss: 1.134089
Train Epoch: 0 [38400/50000 (77%)]	Loss: 1.001651
Train Epoch: 0 [44800/50000 (90%)]	Loss: 0.913920

Test set: Average loss: 1.3661, Accuracy: 5617/10000 (56%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.624488
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.744100
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.960869
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.753951
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.656955
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.703311
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.774766
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.660900

Test set: Average loss: 0.7522, Accuracy: 7438/10000 (74%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.507769
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.516

In [0]:
def test_acc(model):
  val_loader = torch.utils.data.DataLoader(
      datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))
                     ])),
      batch_size=64, shuffle=True, **kwargs)
  model.to_train = False
  t0 = time.time()
  acc = test(model, device, val_loader)
  t1 = time.time()
  model.to_train = True
  print("Inference time ", t1 - t0)
  print("Accuracy", acc)
  return acc

def retrain(model):
  for epoch in range(epoch_range):
      model.to_train = True 
      train(model, device, train_loader, optimizer, epoch)
      model.to_train = False 
      test(model, device, test_loader)
      model.to_train = True
  return test_acc(model)

def prune_loop(model, thresholds, sacrifice):
  init_model_acc = test_acc(model)

  for thres in thresholds:
    print("***** THRES = ", thres, " *****)")
    new_model = Pruner(model, thres = thres, function = "corrs")
    new_model.to_train = False
    acc = test(new_model, device, val_loader)
    new_model.prune()
    new_model.to_train = True
    new_model_acc = retrain(new_model)
    
    i = 0
    while(new_model_acc <= init_model_acc - sacrifice and i < 3):
      print("--- accuracy drop ", i, " ---")
      new_model_acc = retrain(new_model)
      i += 1

    if(new_model_acc <= init_model_acc - sacrifice):
      return model
    model = new_model.model
    print("Number of parameters", sum(p.numel() for p in model.parameters()))
  return model

def add_loop(model,thresholds, sacrifice):
  init_model_acc = test_acc(model)

  for thres in thresholds:
    print("***** THRES = ", thres, " *****)")
    new_model = Adder(model, thres = thres, function = "var")
    new_model.to_train = False
    acc = test(new_model, device, val_loader)
    new_model.add()
    print(new_model)
    new_model.to_train = True
    new_model_acc = retrain(new_model)
    
    i = 0
    while(new_model_acc <= init_model_acc - sacrifice and i < 3):
      print("--- accuracy drop ", i, " ---")
      new_model_acc = retrain(new_model)
      i += 1

    if(new_model_acc <= init_model_acc - sacrifice):
      return model
    model = new_model.model
    print("Number of parameters", sum(p.numel() for p in model.parameters()))
  return model
  
sacrifice = 0.01
thresholds = [.001, .02, .06, .1]
new_model = add_loop(model, thresholds, sacrifice)
    


Test set: Average loss: 0.7188, Accuracy: 7760/10000 (78%)

('Inference time ', 3.83154296875)
('Accuracy', 0.776)
('***** THRES = ', 0.001, ' *****)')

Test set: Average loss: 0.7188, Accuracy: 7760/10000 (78%)

Adder(
  (model): Net(
    (pruning_layers): Sequential(
      (Layer1): Sequential(
        (0): Conv2d(3, 113, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Layer2): Sequential(
        (0): Conv2d(113, 198, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Layer4): Sequential(
        (0): Conv2d(198, 376, kernel_size=(5, 5), stride=(1, 1), p